In [2]:
import pytorch_lightning as pl
import argparse
import os
import pickle
import naturalproofs.dataloaders as dataloaders
import naturalproofs.model as mutils
import torch
import transformers
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
import naturalproofs.model_bert_no_training as model_bert
import json

In [3]:
# use default bert (bert-base-cased)
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-cased')

In [4]:
# load model
model = mutils.Classifier.load_from_checkpoint('ckpt\pairwise_proofwiki.ckpt')
# import naturalproofs.model_bert_no_training as model_bert
# tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-cased')
# model = model_bert.Classifier(
#     pad_idx=tokenizer.pad_token_id,
#     model_type='bert-base-cased',
#     aggregate='avg'
# )

In [5]:
print("Loading data")
ds_raw = pickle.load(open('data\pairwise_proofwiki__bert-base-cased.pkl', 'rb'))

Loading data


In [6]:
xdl, rdl, x2rs = dataloaders.get_eval_dataloaders(
        ds_raw,
        pad=tokenizer.pad_token_id,
        token_limit=8192,
        workers=0,
        split_name='test'
    )
print("%d examples\t%d refs" % (len(xdl.dataset.data), len(rdl.dataset.data)))

ref_dl = rdl
ex_dl = xdl

44 batches (1135 x)
1195 batches (30671 r)
1135 examples	30671 refs


In [7]:
model.eval()
model.cuda()
torch.set_grad_enabled(False)
r_encs, rids = model.pre_encode_refs(ref_dl, progressbar=True)

Pre-encoding references...


100%|██████████| 1195/1195 [03:21<00:00,  5.93it/s]


In [10]:
# prepare data (see from analyze.py)
print("Loading data")
raw_ds = json.load(open('.\\data\\naturalproofs_proofwiki.json', 'r'))
global refs
refs = raw_ds['dataset']['theorems'] + raw_ds['dataset']['definitions'] + raw_ds['dataset']['others']
global id2ref
id2ref = {ref['id'] : ref for ref in refs}

Loading data


In [41]:
標題="Pythagoras's Theorem"
畢達哥拉斯 = """
Let $\triangle ABC$ be a [[Definition:Right Triangle|right triangle]] with $c$ as the [[Definition:Hypotenuse|hypotenuse]].

Then:
:$a^2 + b^2 = c^2$
"""
inputs = "%s%s%s" % (
    標題,
    tokenizer.sep_token,
    畢達哥拉斯
)

ids = tokenizer.encode(inputs)
x = torch.tensor([ids])
x = x.cuda()
x_enc = model.encode_x(x) # (B, D)
# given x theorem, and many references, get the reference rank
logits = model.forward_clf(x_enc, r_encs) # (B, R)

x2ranked = defaultdict(list)
for b in range(logits.size(0)):
    ranked = list(zip(logits[b].tolist(), rids.tolist()))
    ranked = sorted(ranked, reverse=True)
    x2ranked[xid[b].item()] = ranked

    print("====> 相關定理:")
    for top5 in range(5):
        related_rid = ranked[top5][1]
        print(id2ref[related_rid]['title'])
    
    print("====> 實際相關定理:")
    for r in id2ref[3]['refs']:
        print("====> 定理: " + r)



====> 相關定理:
Equivalence of Definitions of Sine and Cosine
Law of Cosines/Right Triangle
Definition:Triangle (Geometry)/Right-Angled
Construction of Square on Given Straight Line
Definition:Cosine
====> 實際相關定理:
====> 定理: Definition:Triangle (Geometry)/Right-Angled
====> 定理: Definition:Triangle (Geometry)/Right-Angled/Hypotenuse


In [40]:
id2ref[3].keys()
id2ref[3]['refs']

['Definition:Triangle (Geometry)/Right-Angled',
 'Definition:Triangle (Geometry)/Right-Angled/Hypotenuse']

In [58]:
畢達哥拉斯 = r"""
The '''predecessor function''' $\operatorname{pred}: \N \to \N$ defined as:
:$\map {\operatorname{pred} } n = \begin{cases}
0 & : n = 0 \\
n-1 & : n > 0
\end{cases}$
is [[Definition:Primitive Recursive Function|primitive recursive]].
"""

inputs = 畢達哥拉斯

ids = tokenizer.encode(inputs)
x = torch.tensor([ids])
x = x.cuda()
x_enc = model.encode_x(x) # (B, D)
# given x theorem, and many references, get the reference rank
logits = model.forward_clf(x_enc, r_encs) # (B, R)

for b in range(logits.size(0)):
    ranked = list(zip(logits[b].tolist(), rids.tolist()))
    ranked = sorted(ranked, reverse=True)

    print("====> 相關定理:")
    for top5 in range(5):
        related_rid = ranked[top5][1]
        print(id2ref[related_rid]['title'])




====> 相關定理:
Definition:Basic Primitive Recursive Function/Successor Function
Predecessor Function is Primitive Recursive
Recursive Function uses One Minimization
Definition:Primitive Recursion/Partial Function
Definition:Basic Primitive Recursive Function


In [ ]:
# Refer to pg4, and pg7 see that ranked is mean listing thm that it related